На клиенте должны быть реализованы следующие функции:
* Код вызова последовательного вызова обучения как минимум двух (N) различных моделей с таким набором данных и параметрами, чтобы обучение одной модели длилось не менее 60 секунд
* Код вызова асинхронного вызова обучения как минимум двух различных моделей с демонстрацией, что работа выполняется в два (в N) раза быстрее
* Асинхронный вызов нескольких предсказаний
* Код демонстрации остальных функций сервера (загрузка, выгрузка, удаление)
* Должны обрабатываться ошибки и исключения, возвращаемые сервером

In [1]:
!pip install aiohttp

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import aiohttp
import asyncio
from aiohttp import ClientSession, web
# import timeit
import time
import json

class ClientException(BaseException):
    def __init__(self, m):
        self.message = m
    # def __str__(self):
    #     return self.message

async def check_error_in_response(response):
    if response.status == 422:
        print("\033[95mClientException:", json.loads(await response.text()).get('detail'), '\033[0m')
        raise ClientException(json.loads(await response.text()).get('detail'))

    return False

async def fetch_data(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            try:
                await check_error_in_response(response)
                r = await response.text()
                return r
            except ClientException as e:
                raise e

async def post_data(url, data):
    async with aiohttp.ClientSession() as session:
        async with session.post(url, json=data) as response:
            try:
                await check_error_in_response(response)
                r = await response.text()
                return json.loads(r)
            except ClientException as e:
                raise e

async def delete_data(url, data):
    async with aiohttp.ClientSession() as session:
        async with session.delete(url, json=data) as response:
            try:
                await check_error_in_response(response)
                r = await response.text()
                return r
            except ClientException as e:
                raise e

In [4]:
BASE_URL = 'http://127.0.0.1:8000/api/v1/models'

In [5]:
logisticRegressionFitReq_body = {
  "X": [
    [1, 2],
    [3, 4]
  ],
  "config": {
    "hyperparameters": {
      "penalty": "l2",
      "dual": False,
      "tol": 0.0001,
      "C": 1.0,
      "fit_intercept": True,
      "intercept_scaling": 1,
      "class_weight": None,
      "random_state": None,
      "solver": "lbfgs",
      "max_iter": 100,
      "multi_class": "auto",
      "verbose": 0,
      "warm_start": False,
      "n_jobs": None
    },
    "id": "logistic_111",
    "ml_model_type": "logistic"
  },
  "y": [5, 6]
}

linearRegressionFitReq_body = {
  "X": [
    [1, 2],
    [3, 4]
  ],
  "config": {
    "hyperparameters": {
      "fit_intercept": True
    },
    "id": "linear_123",
    "ml_model_type": "linear"
  },
  "y": [5, 6]
}

Код вызова последовательного вызова обучения как минимум двух (N) различных моделей с таким набором данных и параметрами, чтобы обучение одной модели длилось не менее 60 секунд

In [9]:
# %%timeit
async def test_fit_model_sync():
    start_time = time.time()

    url = f"{BASE_URL}/fit"

    try:
        linear = await post_data(url, linearRegressionFitReq_body)
        print('linear:', linear.get('message'))

        logistic = await post_data(url, logisticRegressionFitReq_body)
        print('logistic:', logistic.get('message'))
    except Exception as error:
        raise error

    elapsed_time = time.time() - start_time
    print(f"execution time: {elapsed_time:.2f} seconds")

asyncio.run(test_fit_model_sync())

linear: Model 'linear_123' trained and saved.
logistic: Model 'logistic_111' trained and saved.
execution time: 4.02 seconds


Код вызова асинхронного вызова обучения как минимум двух различных моделей с демонстрацией, что работа выполняется в два (в N) раза быстрее

In [18]:
async def test_remove_all():
    try:
        response = await delete_data(f"{BASE_URL}/remove_all", None)
        print('response:', response)
    except ClientException as error:
        error = error.args[0]
        print('\033[94m| Handled Error:')
        print('------------------------------')
        print('| msg:', error, end='\n==============================\n\033[0m\n')

asyncio.run(test_remove_all())

response: {"message":"All models removed."}


In [19]:
# %%timeit
async def test_fit_model_async():
    start_time = time.time()

    url = f"{BASE_URL}/fit"

    try:
        linear, logistic = await asyncio.gather(
            post_data(url, linearRegressionFitReq_body),
            post_data(url, logisticRegressionFitReq_body))

        print('linear:', linear.get('message'))
        print('logistic:', logistic.get('message'))
    except ClientException as error:
        error = error.args[0]
        print('\033[94m| Handled Error:')
        print('------------------------------')
        print('| msg:', error, end='\n==============================\n\033[0m\n')

    elapsed_time = time.time() - start_time
    print(f"execution time: {elapsed_time:.2f} seconds")

asyncio.run(test_fit_model_async())

linear: Model 'linear_123' trained and saved.
logistic: Model 'logistic_111' trained and saved.
execution time: 2.02 seconds


Асинхронный вызов нескольких предсказаний

In [27]:
async def test_load():
    try:
        linear, logistic = await asyncio.gather(
            post_data(f"{BASE_URL}/load", {"id": "linear_123"}),
            post_data(f"{BASE_URL}/load", {"id": "logistic_111"}),
        )

        print('linear:', linear.get('message'))
        print('logistic:', logistic.get('message'))
    except ClientException as error:
        error = error.args[0]
        print('\033[94m| Handled Error:')
        print('------------------------------')
        print('| msg:', error, end='\n==============================\n\033[0m\n')

asyncio.run(test_load())


linear: Model 'linear_123' loaded for inference.
logistic: Model 'logistic_111' loaded for inference.


In [21]:
async def test_predict_async():
    start_time = time.time()
    url = f"{BASE_URL}/predict"

    try:
        linear, logistic = await asyncio.gather(
            post_data(url, {
                "id": "linear_123",
                "X": [
                    [7, 8]
                ]
            }),
            post_data(url, {
                "id": "logistic_111",
                "X": [
                    [7, 8]
                ]
            }))

        print('linear:', linear.get('predictions'))
        print('logistic:', logistic.get('predictions'))
    except ClientException as error:
        error_args = error.args

        for arg in range(len(error_args)):
            error = error_args[arg]
            print('\033[94m| Handled Error:')
            print('------------------------------')
            print('| msg:', error, end='\n==============================\n\033[0m\n')

    elapsed_time = time.time() - start_time
    print(f"execution time: {elapsed_time:.2f} seconds")

asyncio.run(test_predict_async())

ClientException: No models loaded. 
ClientException: No models loaded. 
| Handled Error:
------------------------------
| msg: No models loaded.

execution time: 0.01 seconds


Код демонстрации остальных функций сервера (загрузка, выгрузка, удаление)

In [31]:
async def test_load_async():
    print('===>> test_load_async')
    try:
        await asyncio.gather(
            post_data(f"{BASE_URL}/load", {"id": "linear_123"}),
            post_data(f"{BASE_URL}/load", {"id": "logistic_111"}),
        )
        print('==>> УСПЕШНО')
    except ClientException as error:
        error_args = error.args
        print('\033[94m| Handled Error:')
        print('------------------------------')
        print('| msg:', error_args[0], end='\n==============================\n\033[0m\n')
        print('==>> ПРОВАЛ')

async def test_get_status():
    print('===>> test_get_status')
    try:
        response = await fetch_data(f"{BASE_URL}/get_status")
        print('response:', response)
        print('==>> УСПЕШНО')
    except ClientException as error:
        print('==>> ПРОВАЛ')

async def test_list_models():
    print('===>> test_list_models')
    try:
        response = await fetch_data(f"{BASE_URL}/list_models")
        print('response:', response)
        print('==>> УСПЕШНО')
    except ClientException as error:
        print('==>> ПРОВАЛ')

async def test_unload_async():
    print('===>> test_unload_async')
    try:
        await asyncio.gather(
            post_data(f"{BASE_URL}/unload", {"id": "linear_123"}),
            post_data(f"{BASE_URL}/unload", {"id": "logistic_111"}),
        )
        print('==>> УСПЕШНО')
    except ClientException as error:
        print('te st_unload_async error:', error, type(error))
        error_args = error.args
        print('\033[94m| Handled Error:')
        print('------------------------------')
        print('| msg:', error_args[0], end='\n==============================\n\033[0m\n')
        print('==>> ПРОВАЛ')

async def test_delete_model(model_id='logistic_111'):
    print('===>> test_delete_model')
    try:
        response = await delete_data(f"{BASE_URL}/remove/{model_id}", None)
        print('response:', response)
        print('==>> УСПЕШНО')
    except ClientException as error:
        print('test_delete_model error:', error, type(error))
        print('==>> ПРОВАЛ')


# Удаляем все модели из хранилища и из инференса
asyncio.run(test_remove_all())

# Обучаем 2 разные модели асинхронно - они сохраняются в хранилище
asyncio.run(test_fit_model_async())

# Загружаем обе модели на инференс
asyncio.run(test_load_async())

# Проверяем статус - обе модели должны отображаться в списке моделей
asyncio.run(test_get_status())

# Выгрузим модели из инференса асинхронно
asyncio.run(test_unload_async())

# Проверим статус, сервер должен ответить, что ничего не загружено
asyncio.run(test_get_status())

# Посмотрим, какие модели доступны в хранилище для загрузки на инференс
asyncio.run(test_list_models())

# Загрузим обе модели обратно из хранилища на инференс
asyncio.run(test_load_async())

# Удалим одну модель c id = logistic_111.
# Она будет удалена из инференса и из хранилища
asyncio.run(test_delete_model(model_id='logistic_111'))

# Проверим, что модели нет в списке загруженных на инференс
asyncio.run(test_get_status())

# Проверим, что модели нет в хранилище
asyncio.run(test_list_models())

# Удалим все модели из инференса и из хранилища
asyncio.run(test_remove_all())

response: {"message":"All models removed."}
linear: Model 'linear_123' trained and saved.
logistic: Model 'logistic_111' trained and saved.
execution time: 4.03 seconds
===>> test_load_async
ClientException: Model already loaded. 
| Handled Error:
------------------------------
| msg: Model already loaded.

==>> ПРОВАЛ
===>> test_get_status
response: {"status":"Models: ['linear_123']"}
==>> УСПЕШНО
===>> test_unload_async
ClientException: Model with id 'logistic_111' was not loaded. 
te st_unload_async error: Model with id 'logistic_111' was not loaded. <class '__main__.ClientException'>
| Handled Error:
------------------------------
| msg: Model with id 'logistic_111' was not loaded.

==>> ПРОВАЛ
===>> test_get_status
response: {"status":"No models loaded."}
==>> УСПЕШНО
===>> test_list_models
response: {"models":["logistic_111","linear_123"]}
==>> УСПЕШНО
===>> test_load_async
==>> УСПЕШНО
===>> test_delete_model
response: {"message":"Model 'logistic_111' removed"}
==>> УСПЕШНО
===>

In [29]:
async def test_load_sync():
    print('===>> test_load_sync')
    try:
        await post_data(f"{BASE_URL}/load", {"id": "linear_123"}),
        print('==>> УСПЕШНО')
    except ClientException as error:
        error_args = error.args
        print('\033[94m| Handled Error:')
        print('------------------------------')
        print('| msg:', error_args[0], end='\n==============================\n\033[0m\n')
        print('==>> ПРОВАЛ')

async def test_get_status():
    print('===>> test_get_status')
    try:
        response = await fetch_data(f"{BASE_URL}/get_status")
        print('response:', response)
        print('==>> УСПЕШНО')
    except ClientException as error:
        print('==>> ПРОВАЛ')

async def test_list_models():
    print('===>> test_list_models')
    try:
        response = await fetch_data(f"{BASE_URL}/list_models")
        print('response:', response)
        print('==>> УСПЕШНО')
    except ClientException as error:
        print('==>> ПРОВАЛ')

async def test_unload_sync():
    print('===>> test_unload_sync')
    try:
        await post_data(f"{BASE_URL}/unload", {"id": "linear_123"}),
        print('==>> УСПЕШНО')
    except ClientException as error:
        print('te st_unload_async error:', error, type(error))
        error_args = error.args
        print('\033[94m| Handled Error:')
        print('------------------------------')
        print('| msg:', error_args[0], end='\n==============================\n\033[0m\n')
        print('==>> ПРОВАЛ')

async def test_delete_model(model_id='logistic_111'):
    print('===>> test_delete_model')
    try:
        response = await delete_data(f"{BASE_URL}/remove/{model_id}", None)
        print('response:', response)
        print('==>> УСПЕШНО')
    except ClientException as error:
        print('test_delete_model error:', error, type(error))
        print('==>> ПРОВАЛ')


# Удаляем все модели из хранилища и из инференса
asyncio.run(test_remove_all())

# Обучаем 2 разные модели асинхронно - они сохраняются в хранилище
asyncio.run(test_fit_model_async())

# Загружаем обе модели на инференс
asyncio.run(test_load_sync())

# Проверяем статус - обе модели должны отображаться в списке моделей
asyncio.run(test_get_status())

# Выгрузим модели из инференса асинхронно
asyncio.run(test_unload_sync())

# Проверим статус, сервер должен ответить, что ничего не загружено
asyncio.run(test_get_status())

# Посмотрим, какие модели доступны в хранилище для загрузки на инференс
asyncio.run(test_list_models())

# Загрузим обе модели обратно из хранилища на инференс
asyncio.run(test_load_sync())

# Удалим одну модель c id = logistic_111.
# Она будет удалена из инференса и из хранилища
asyncio.run(test_delete_model(model_id='logistic_111'))

# Проверим, что модели нет в списке загруженных на инференс
asyncio.run(test_get_status())

# Проверим, что модели нет в хранилище
asyncio.run(test_list_models())

# Удалим все модели из инференса и из хранилища
asyncio.run(test_remove_all())

response: {"message":"All models removed."}
linear: Model 'linear_123' trained and saved.
logistic: Model 'logistic_111' trained and saved.
execution time: 4.02 seconds
===>> test_load_sync
==>> УСПЕШНО
===>> test_get_status
response: {"status":"Models: ['linear_123']"}
==>> УСПЕШНО
===>> test_unload_sync
==>> УСПЕШНО
===>> test_get_status
response: {"status":"No models loaded."}
==>> УСПЕШНО
===>> test_list_models
response: {"models":["logistic_111","linear_123"]}
==>> УСПЕШНО
===>> test_load_sync
==>> УСПЕШНО
===>> test_delete_model
response: {"message":"Model 'logistic_111' removed"}
==>> УСПЕШНО
===>> test_get_status
response: {"status":"Models: ['linear_123']"}
==>> УСПЕШНО
===>> test_list_models
response: {"models":["linear_123"]}
==>> УСПЕШНО
response: {"message":"All models removed."}


**Пояснение к работе**
Обработка ошибок происходит на двух уровнях на клиенте - на уровне получения данных, где создаётся исключение ClientException (в output подсвечен розовым цветом) - общий обработчик для всех ошибок (для того, чтобы клиентское приложение не падало при неопознанной ошибке при ответе от серевера, который не ожидался клиентом. Далее, на уровне логики исключения отлавливают уже обработчики, которые обрабатывают их по своей логике (в output подсвечены синим цветом). Если бы это был бы полноценный клиент, например, веб-приложению, то мы бы отобразили пользователю ошибку в виде всплывающего окна или уведомления. Поскольку API ответы не стандартизированы, то записать код только в один обработчик невозможно, поскольку каждый ответ с кодом 422 немного отличается по структуре.
В итоге, если полученная ошибка не будет обработана в логике, она всегда будет обработана на уровне работы с данными при помощи ClientException.